In [ ]:
import sys

from wrappers import (
    ActionMaskInInfoWrapper,
    ChannelLastToFirstWrapper,
    FrameStackWrapper,
    TwoPlayerPlayerPlaneWrapper,
)


sys.path.append("../..")
from agent_configs import MuZeroConfig
import gymnasium as gym
from utils.utils import CategoricalCrossentropyLoss
from action_functions import action_as_plane, action_as_onehot
from muzero_agent_torch import MuZeroAgent
from pettingzoo.classic import tictactoe_v3
from game_configs import TicTacToeConfig, CartPoleConfig
from utils import MSELoss
import torch
import os
from torch.optim import Adam, SGD
from agents.random import RandomAgent
from agents.tictactoe_expert import TicTacToeBestAgent
from supersuit import frame_stack_v1, agent_indicator_v0

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)


config = {
    "known_bounds": [0, 500],
    "residual_layers": [],  # ??? more depth better? up to depth 16, need at most 16 filters
    "representation_dense_layer_widths": [512, 64],
    "dynamics_dense_layer_widths": [512, 64],
    "actor_conv_layers": [],  # ???
    "critic_conv_layers": [],  # ???
    "reward_conv_layers": [],
    "actor_dense_layer_widths": [512],  # ???
    "critic_dense_layer_widths": [512],  # ???
    "reward_dense_layer_widths": [],
    "conv_layers": [],
    "dense_layer_widths": [],
    "noisy_sigma": 0.0,
    "value_loss_factor": 1.0,
    "root_dirichlet_alpha": 0.25,  # ???
    "root_exploration_fraction": 0.25,
    "num_simulations": 50,  # ??? goal is to increase this and see if it learns faster
    "temperatures": [1.0, 0.5, 0.25],
    "temperature_updates": [30000, 60000],
    "temperature_with_training_steps": True,
    "clip_low_prob": 0.0,
    "pb_c_base": 19652,
    "pb_c_init": 1.25,
    "optimizer": Adam,
    "learning_rate": 0.005,  # ??? find a learning rate that works okay (no exploding, but not too small) # 0.1 to 0.01 decrease to 10% of the init value after 400k steps in pseudocode, but 0.2 in alphazero paper (and decreased 3 times)
    "momentum": 0.9,
    "adam_epsilon": 1e-8,
    "discount_factor": 0.997,
    "value_loss_function": CategoricalCrossentropyLoss(),
    "reward_loss_function": CategoricalCrossentropyLoss(),
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "action_function": action_as_onehot,
    "training_steps": 100000,
    "minibatch_size": 128,  # ??? this should be about 0.1 of the number of positions collected... or is it in the replay buffer? AlphaZero did a batch size of 4096 muzero 2048, and they said this was about 0.1.
    "min_replay_buffer_size": 5000,  # ???
    "replay_buffer_size": 50000,  # ??? paper used a buffer size of 1M games
    "unroll_steps": 5,
    "n_step": 10,
    "clipnorm": 0.0,
    "weight_decay": 0.0001,
    "kernel_initializer": "he_normal",  # ???
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "per_use_batch_weights": True,
    "per_initial_priority_max": True,
    "per_epsilon": 0.0001,
    "multi_process": True,
    "num_workers": 6,  # ???
    "lr_ratio": float("inf"),  # 0.1
    # "lr_ratio": 0.1,  # 0.1
    "games_per_generation": 8,  # ??? AlphaZero did ~64 games per generation
    "reanalyze": True,  # TODO
    "support_range": 31,
}

# DO AN ABALATION ON NUM SIMULATIONS, THE OG PAPER FOUND MORE SIMULATIONS MEANS BETTER LEARNING SIGNAL
# CHECK MY MCTS STUFF, IS THE SIGN CORRECT? IS IT CORRECT WITH REWARDS? IS IT CORRECT FOR TERMINAL STATES?

# steps, run tictactoe on fast settings for at least 200k steps, see if it learns to play okay
# add only updating mcts network every x steps
# add a ratio for learning steps to self play steps
# # run it without multiprocessing
# increase num simulations to 50 or 100 and see if it learns faster


env = CartPoleConfig().make_env()
game_config = CartPoleConfig()
config = MuZeroConfig(config, game_config)

agent = MuZeroAgent(
    env,
    config,
    name="muzero_cartpole",
    device="cpu",
    test_agents=[],
)

In [ ]:
agent.checkpoint_interval = 10
agent.test_interval = 250
agent.test_trials = 50
agent.train()

In [ ]:
import sys


sys.path.append("../..")
from agent_configs import MuZeroConfig
import gymnasium as gym
from utils.utils import CategoricalCrossentropyLoss
from action_functions import action_as_plane
from muzero_agent_torch import MuZeroAgent
from pettingzoo.classic import tictactoe_v3
from game_configs import TicTacToeConfig, CartPoleConfig
from utils import MSELoss
import torch
import os
from torch.optim import Adam, SGD
from agents.random import RandomAgent
from agents.tictactoe_expert import TicTacToeBestAgent

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)


config = {
    "known_bounds": [-1, 1],
    "residual_layers": [(24, 3, 1)] * 1,  # increase num layers
    "conv_layers": [],
    "dense_layers": [],
    "actor_conv_layers": [(16, 1, 1)],
    "critic_conv_layers": [(16, 1, 1)],
    "reward_conv_layers": [(16, 1, 1)],
    "actor_dense_layer_widths": [],
    "critic_dense_layer_widths": [],
    "reward_dense_layer_widths": [],
    "conv_layers": [],
    "dense_layer_widths": [],
    "noisy_sigma": 0.0,
    "value_loss_factor": 1.0,
    "root_dirichlet_alpha": 0.25,
    "root_exploration_fraction": 0.25,
    "num_simulations": 25,  # try larger
    "temperatures": [1.0, 0.1],
    "temperature_updates": [8],  # change this
    "temperature_with_training_steps": False,
    "clip_low_prob": 0.0,
    "pb_c_base": 19652,
    "pb_c_init": 1.25,
    "optimizer": Adam,
    "learning_rate": 0.001,  # slightly increase this, maybe 0.002 or 0.003
    "momentum": 0.0,
    "adam_epsilon": 1e-8,  # try lower
    "value_loss_function": MSELoss(),  #  MSELoss(),
    "reward_loss_function": MSELoss(),  # MSELoss(),
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "action_function": action_as_plane,
    "training_steps": 100000,
    "minibatch_size": 8,
    "min_replay_buffer_size": 4000,  # try lower (or just different)
    "replay_buffer_size": 100000,  # try lower, 50k or 20k or 10k
    "unroll_steps": 5,
    "n_step": 9,
    "clipnorm": 0.0,
    "weight_decay": 0.0001,
    "kernel_initializer": "glorot_normal",  # try different
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,  # 0.0 was original
    "per_use_batch_weights": False,
    "per_initial_priority_max": True,
    "per_epsilon": 0.0001,
    "multi_process": True,
    "num_workers": 2,
    "reanalyze": True,  # TODO
    "support_range": None,  # None
}

# REANALYZE NOT IMPLEMENTED BUT NEVER USED IN THING I SAW ONLINE (like it is computed but never ends up used since bootstrap index always > than len of game history)

# add a max depth to the tree
# game priority is max of position priorities of the game
# with these two changes should be an exact match with online github implementation

env = tictactoe_v3.env(render_mode="rgb_array")
env = ActionMaskInInfoWrapper(env)
env = FrameStackWrapper(env, 4, channel_first=False)
env = TwoPlayerPlayerPlaneWrapper(env, channel_first=False)
env = ChannelLastToFirstWrapper(env)

game_config = TicTacToeConfig()
config = MuZeroConfig(config, game_config)

agent = MuZeroAgent(
    env,
    config,
    name="muzero_tictactoe-hyperopt-best",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)

In [ ]:
agent.checkpoint_interval = 50
agent.test_interval = 1000
agent.test_trials = 300
agent.train()

In [ ]:
import sys

from wrappers import (
    ActionMaskInInfoWrapper,
    ChannelLastToFirstWrapper,
    FrameStackWrapper,
    TwoPlayerPlayerPlaneWrapper,
)


sys.path.append("../..")
from agent_configs import MuZeroConfig
import gymnasium as gym
from utils.utils import CategoricalCrossentropyLoss
from action_functions import action_as_plane, action_as_onehot
from muzero_agent_torch import MuZeroAgent
from pettingzoo.classic import tictactoe_v3
from game_configs import TicTacToeConfig, CartPoleConfig
from utils import MSELoss
import torch
import os
from torch.optim import Adam, SGD
from agents.random import RandomAgent
from agents.tictactoe_expert import TicTacToeBestAgent
from supersuit import frame_stack_v1, agent_indicator_v0

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)


config = {
    "known_bounds": [-1, 1],
    "residual_layers": [],  # ??? more depth better? up to depth 16, need at most 16 filters
    "representation_dense_layer_widths": [256, 64],
    "dynamics_dense_layer_widths": [256, 64],
    "actor_conv_layers": [],  # ???
    "critic_conv_layers": [],  # ???
    "reward_conv_layers": [],
    "actor_dense_layer_widths": [256],  # ???
    "critic_dense_layer_widths": [256],  # ???
    "reward_dense_layer_widths": [],
    "conv_layers": [],
    "dense_layer_widths": [],
    "noisy_sigma": 0.0,
    "value_loss_factor": 1.0,
    "root_dirichlet_alpha": 0.25,  # ???
    "root_exploration_fraction": 0.25,
    "num_simulations": 25,  # ??? goal is to increase this and see if it learns faster
    "temperatures": [1.0, 0.1],
    "temperature_updates": [5],
    "temperature_with_training_steps": False,
    "clip_low_prob": 0.0,
    "pb_c_base": 19652,
    "pb_c_init": 1.25,
    "optimizer": Adam,
    "learning_rate": 0.002,  # ??? find a learning rate that works okay (no exploding, but not too small) # 0.1 to 0.01 decrease to 10% of the init value after 400k steps in pseudocode, but 0.2 in alphazero paper (and decreased 3 times)
    "momentum": 0.9,
    "adam_epsilon": 1e-8,
    "value_loss_function": MSELoss(),
    "reward_loss_function": MSELoss(),
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "action_function": action_as_onehot,
    "training_steps": 100000,
    "minibatch_size": 128,  # ??? this should be about 0.1 of the number of positions collected... or is it in the replay buffer? AlphaZero did a batch size of 4096 muzero 2048, and they said this was about 0.1.
    "min_replay_buffer_size": 5000,  # ???
    "replay_buffer_size": 20000,  # ??? paper used a buffer size of 1M games
    "unroll_steps": 5,
    "n_step": 9,
    "clipnorm": 0.0,
    "weight_decay": 0.0001,
    "kernel_initializer": "he_normal",  # ???
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "per_use_batch_weights": True,
    "per_initial_priority_max": True,
    "per_epsilon": 0.0001,
    "multi_process": True,
    "num_workers": 6,  # ???
    # "lr_ratio": float("inf"),  # 0.1
    "lr_ratio": 0.1,  # 0.1
    "games_per_generation": 8,  # ??? AlphaZero did ~64 games per generation
    "reanalyze": True,  # TODO
    "support_range": None,
}

# DO AN ABALATION ON NUM SIMULATIONS, THE OG PAPER FOUND MORE SIMULATIONS MEANS BETTER LEARNING SIGNAL
# CHECK MY MCTS STUFF, IS THE SIGN CORRECT? IS IT CORRECT WITH REWARDS? IS IT CORRECT FOR TERMINAL STATES?

# steps, run tictactoe on fast settings for at least 200k steps, see if it learns to play okay
# add only updating mcts network every x steps
# add a ratio for learning steps to self play steps
# # run it without multiprocessing
# increase num simulations to 50 or 100 and see if it learns faster


env = tictactoe_v3.env(render_mode="rgb_array")
env = ActionMaskInInfoWrapper(env)
env = FrameStackWrapper(env, 4, channel_first=False)
env = TwoPlayerPlayerPlaneWrapper(env, channel_first=False)
env = ChannelLastToFirstWrapper(env)

game_config = TicTacToeConfig()
config = MuZeroConfig(config, game_config)

agent = MuZeroAgent(
    env,
    config,
    name="muzero_tictactoe-dense-5moves_1",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)

tested the ones i found on the github and they worked good? but i think i am gonna do less workers because they did 480k episodes after 32k training steps, i got 250k after like 5k steps, so i want to change that

In [ ]:
agent.checkpoint_interval = 100
agent.test_interval = 250
agent.test_trials = 100
agent.train()

In [ ]:
import sys

from wrappers import (
    ActionMaskInInfoWrapper,
    ChannelLastToFirstWrapper,
    FrameStackWrapper,
    TwoPlayerPlayerPlaneWrapper,
)


sys.path.append("../..")
from agent_configs import MuZeroConfig
import gymnasium as gym
from utils.utils import CategoricalCrossentropyLoss
from action_functions import action_as_plane, action_as_onehot
from muzero_agent_torch import MuZeroAgent
from pettingzoo.classic import tictactoe_v3
from game_configs import TicTacToeConfig, CartPoleConfig
from utils import MSELoss
import torch
import os
from torch.optim import Adam, SGD
from agents.random import RandomAgent
from agents.tictactoe_expert import TicTacToeBestAgent
from supersuit import frame_stack_v1, agent_indicator_v0

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)


config = {
    "known_bounds": [-1, 1],
    "residual_layers": [(16, 3, 1)],
    "representation_dense_layer_widths": [],
    "dynamics_dense_layer_widths": [],
    "actor_conv_layers": [(8, 1, 1)],  # ???
    "critic_conv_layers": [(8, 1, 1)],  # ???
    "reward_conv_layers": [],
    "actor_dense_layer_widths": [],  # ???
    "critic_dense_layer_widths": [],  # ???
    "reward_dense_layer_widths": [],
    "conv_layers": [],
    "dense_layer_widths": [],
    "noisy_sigma": 0.0,
    "value_loss_factor": 1.0,
    "root_dirichlet_alpha": 1.8,  # ???
    "root_exploration_fraction": 0.25,
    "num_simulations": 25,  # ??? goal is to increase this and see if it learns faster
    "temperatures": [1.0, 0.1],
    "temperature_updates": [2],
    "temperature_with_training_steps": False,
    "clip_low_prob": 0.0,
    "pb_c_base": 19652,
    "pb_c_init": 1.25,
    "optimizer": SGD,
    "learning_rate": 0.1,  # ??? find a learning rate that works okay (no exploding, but not too small) # 0.1 to 0.01 decrease to 10% of the init value after 400k steps in pseudocode, but 0.2 in alphazero paper (and decreased 3 times)
    "momentum": 0.9,
    "adam_epsilon": 1e-8,
    "value_loss_function": MSELoss(),
    "reward_loss_function": MSELoss(),
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "action_function": action_as_plane,
    "training_steps": 4000,
    "minibatch_size": 32,  # ??? this should be about 0.1 of the number of positions collected... or is it in the replay buffer? AlphaZero did a batch size of 4096 muzero 2048, and they said this was about 0.1.
    "min_replay_buffer_size": 1000,  # 9000 # ???
    "replay_buffer_size": 10000,  # ??? paper used a buffer size of 1M games
    "unroll_steps": 5,
    "n_step": 9,
    "clipnorm": 1.0,
    "weight_decay": 0.0001,
    "kernel_initializer": "orthogonal",  # ???
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "per_use_batch_weights": False,
    "per_initial_priority_max": False,
    "per_epsilon": 0.0001,
    "multi_process": True,
    "num_workers": 2,  # ???
    "lr_ratio": float("inf"),
    # "lr_ratio": 0.1,
    "games_per_generation": 8,  # ??? AlphaZero did ~64 games per generation
    "reanalyze": True,  # TODO
    "support_range": None,
}

# DO AN ABALATION ON NUM SIMULATIONS, THE OG PAPER FOUND MORE SIMULATIONS MEANS BETTER LEARNING SIGNAL
# CHECK MY MCTS STUFF, IS THE SIGN CORRECT? IS IT CORRECT WITH REWARDS? IS IT CORRECT FOR TERMINAL STATES?

# steps, run tictactoe on fast settings for at least 200k steps, see if it learns to play okay
# add only updating mcts network every x steps
# add a ratio for learning steps to self play steps
# add frame stacking
# run it without multiprocessing
# increase num simulations to 50 or 100 and see if it learns faster


env = tictactoe_v3.env(render_mode="rgb_array")
env = ActionMaskInInfoWrapper(env)
env = FrameStackWrapper(env, 4, channel_first=False)
env = TwoPlayerPlayerPlaneWrapper(env, channel_first=False)
env = ChannelLastToFirstWrapper(env)

game_config = TicTacToeConfig()
config = MuZeroConfig(config, game_config)

agent = MuZeroAgent(
    env,
    config,
    name="muzero_tictactoe_hyperopt",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],
)

with 2 moves left one losing one drawing, it finds the drawing move, but evaluates the winning position after tree search as 0 instead of 1 (though predicts it as 1). shown in paper 2m-3

In [ ]:
agent.checkpoint_interval = 100
agent.test_interval = 250
agent.test_trials = 100
agent.train()

In [ ]:
import torch
from packages.utils.utils.utils import process_petting_zoo_obs

from pettingzoo.classic import tictactoe_v3


def play_game(player1, player2):

    env = tictactoe_v3.env(render_mode="rgb_array")
    with torch.no_grad():  # No gradient computation during testing
        # Reset environment
        env.reset()
        state, reward, termination, truncation, info = env.last()
        done = termination or truncation
        agent_id = env.agent_selection
        current_player = env.agents.index(agent_id)
        state, info = process_petting_zoo_obs(state, info, current_player)
        agent_names = env.agents.copy()

        episode_length = 0
        while not done and episode_length < 1000:  # Safety limit
            # Get current agent and player
            episode_length += 1

            # Get action from average strategy
            if current_player == 0:
                prediction = player1.predict(state, info, env=env)
                action = player1.select_actions(prediction, info).item()
            else:
                prediction = player2.predict(state, info, env=env)
                action = player2.select_actions(prediction, info).item()

            # Step environment
            env.step(action)
            state, reward, termination, truncation, info = env.last()
            agent_id = env.agent_selection
            current_player = env.agents.index(agent_id)
            state, info = process_petting_zoo_obs(state, info, current_player)
            done = termination or truncation
        print(env.rewards)
        return env.rewards["player_0"]

In [ ]:
from agents.random import RandomAgent
from agents.tictactoe_expert import TicTacToeBestAgent
from elo.elo import StandingsTable


random_vs_expert_table = StandingsTable([agent, TicTacToeBestAgent()], start_elo=1400)
random_vs_expert_table.play_1v1_tournament(1000, play_game)
print(random_vs_expert_table.bayes_elo())
print(random_vs_expert_table.get_win_table())

In [ ]:
import numpy as np


class TicTacToeBestAgent:
    def __init__(self, model_name="tictactoe_expert"):
        self.model_name = model_name

    def predict(self, observation, info, env=None):
        return observation, info

    def select_actions(self, prediction, info):
        # Reconstruct board: +1 for current player, -1 for opponent, 0 otherwise
        board = prediction[0][0] - prediction[0][1]
        print(board)
        # Default: random legal move
        action = np.random.choice(info["legal_moves"])

        # Horizontal and vertical checks
        for i in range(3):
            # Row
            if np.sum(board[i, :]) == 2 and 0 in board[i, :]:
                ind = np.where(board[i, :] == 0)[0][0]
                return np.ravel_multi_index((i, ind), (3, 3))
            elif abs(np.sum(board[i, :])) == 2 and 0 in board[i, :]:
                ind = np.where(board[i, :] == 0)[0][0]
                action = np.ravel_multi_index((i, ind), (3, 3))

            # Column
            if np.sum(board[:, i]) == 2 and 0 in board[:, i]:
                ind = np.where(board[:, i] == 0)[0][0]
                return np.ravel_multi_index((ind, i), (3, 3))
            elif abs(np.sum(board[:, i])) == 2 and 0 in board[:, i]:
                ind = np.where(board[:, i] == 0)[0][0]
                action = np.ravel_multi_index((ind, i), (3, 3))

        # Diagonals
        diag = board.diagonal()
        if np.sum(diag) == 2 and 0 in diag:
            ind = np.where(diag == 0)[0][0]
            return np.ravel_multi_index((ind, ind), (3, 3))
        elif abs(np.sum(diag)) == 2 and 0 in diag:
            ind = np.where(diag == 0)[0][0]
            action = np.ravel_multi_index((ind, ind), (3, 3))

        anti_diag = np.fliplr(board).diagonal()
        if np.sum(anti_diag) == 2 and 0 in anti_diag:
            ind = np.where(anti_diag == 0)[0][0]
            return np.ravel_multi_index((ind, 2 - ind), (3, 3))
        elif abs(np.sum(anti_diag)) == 2 and 0 in anti_diag:
            ind = np.where(anti_diag == 0)[0][0]
            action = np.ravel_multi_index((ind, 2 - ind), (3, 3))

        return action

In [ ]:
import sys

from wrappers import (
    ActionMaskInInfoWrapper,
    ChannelLastToFirstWrapper,
    FrameStackWrapper,
    TwoPlayerPlayerPlaneWrapper,
)


sys.path.append("../..")
from agent_configs import MuZeroConfig
import gymnasium as gym
from utils.utils import CategoricalCrossentropyLoss
from action_functions import action_as_plane, action_as_onehot
from muzero_agent_torch import MuZeroAgent
from pettingzoo.classic import tictactoe_v3
from game_configs import TicTacToeConfig, CartPoleConfig
from utils import MSELoss
import torch
import os
from torch.optim import Adam, SGD
from agents.random import RandomAgent
from agents.tictactoe_expert import TicTacToeBestAgent
from supersuit import frame_stack_v1, agent_indicator_v0

# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)


config = {
    "known_bounds": [-1, 1],
    "residual_layers": [(16, 3, 1)],
    "representation_dense_layer_widths": [],
    "dynamics_dense_layer_widths": [],
    "actor_conv_layers": [],  # ???
    "critic_conv_layers": [],  # ???
    "reward_conv_layers": [],
    "actor_dense_layer_widths": [],  # ???
    "critic_dense_layer_widths": [],  # ???
    "reward_dense_layer_widths": [],
    "conv_layers": [],
    "dense_layer_widths": [],
    "noisy_sigma": 0.0,
    "value_loss_factor": 1.0,
    "root_dirichlet_alpha": 2.0,  # ???
    "root_exploration_fraction": 0.25,
    "num_simulations": 25,  # ??? goal is to increase this and see if it learns faster
    "temperatures": [1.0, 0.1],
    "temperature_updates": [5],
    "temperature_with_training_steps": False,
    "clip_low_prob": 0.0,
    "pb_c_base": 19652,
    "pb_c_init": 1.25,
    "optimizer": Adam,
    "learning_rate": 0.001,  # ??? find a learning rate that works okay (no exploding, but not too small) # 0.1 to 0.01 decrease to 10% of the init value after 400k steps in pseudocode, but 0.2 in alphazero paper (and decreased 3 times)
    "momentum": 0.0,
    "adam_epsilon": 1e-8,
    "value_loss_function": MSELoss(),
    "reward_loss_function": MSELoss(),
    "policy_loss_function": CategoricalCrossentropyLoss(),
    "action_function": action_as_plane,
    "training_steps": 33000,
    "minibatch_size": 32,  # ??? this should be about 0.1 of the number of positions collected... or is it in the replay buffer? AlphaZero did a batch size of 4096 muzero 2048, and they said this was about 0.1.
    "min_replay_buffer_size": 1000,  # 9000 # ???
    "replay_buffer_size": 40000,  # ??? paper used a buffer size of 1M games
    "unroll_steps": 5,
    "n_step": 9,
    "clipnorm": 0.0,
    "weight_decay": 0.0001,
    "kernel_initializer": "orthogonal",  # ???
    "per_alpha": 0.0,
    "per_beta": 0.0,
    "per_beta_final": 0.0,
    "per_use_batch_weights": False,
    "per_initial_priority_max": False,
    "per_epsilon": 0.0001,
    "multi_process": True,
    "num_workers": 2,  # ???
    "lr_ratio": float("inf"),
    # "lr_ratio": 0.1,
    "games_per_generation": 8,  # ??? AlphaZero did ~64 games per generation
    "reanalyze": True,  # TODO
    "support_range": None,
}

# DO AN ABALATION ON NUM SIMULATIONS, THE OG PAPER FOUND MORE SIMULATIONS MEANS BETTER LEARNING SIGNAL
# CHECK MY MCTS STUFF, IS THE SIGN CORRECT? IS IT CORRECT WITH REWARDS? IS IT CORRECT FOR TERMINAL STATES?

# steps, run tictactoe on fast settings for at least 200k steps, see if it learns to play okay
# add only updating mcts network every x steps
# add a ratio for learning steps to self play steps
# add frame stacking
# run it without multiprocessing
# increase num simulations to 50 or 100 and see if it learns faster


env = tictactoe_v3.env(render_mode="rgb_array")
env = ActionMaskInInfoWrapper(env)
env = FrameStackWrapper(env, 4, channel_first=False)
env = TwoPlayerPlayerPlaneWrapper(env, channel_first=False)
env = ChannelLastToFirstWrapper(env)

game_config = TicTacToeConfig()
config = MuZeroConfig(config, game_config)

agent = MuZeroAgent(
    env,
    config,
    name="muzero_tictactoe_hyperopt-2",
    device="cpu",
    test_agents=[RandomAgent(), TicTacToeBestAgent()],  # RandomAgent(),
)

agent.checkpoint_interval = 100
agent.test_interval = 1000
agent.test_trials = 500
agent.train()

In [ ]:
agent.config.training_steps += 100
agent.train()

In [ ]:
agent.config.num_simulations = 800

In [ ]:
import sys

sys.path.append("../")
from wrappers import (
    ActionMaskInInfoWrapper,
    ChannelLastToFirstWrapper,
    TwoPlayerPlayerPlaneWrapper,
    FrameStackWrapper,
)
from pettingzoo.classic import tictactoe_v3
from game_configs import TicTacToeConfig

# from agents.tictactoe_expert import TicTacToeBestAgent

# from agents.random import RandomAgent
from supersuit import frame_stack_v1, agent_indicator_v0

best_agent = TicTacToeBestAgent()
env = tictactoe_v3.env(render_mode="rgb_array")
print(env.observation_space("player_0"))
env = ActionMaskInInfoWrapper(env)
print(env.observation_space("player_0"))
env = FrameStackWrapper(env, 4, channel_first=False)
print(env.observation_space("player_0"))
env = TwoPlayerPlayerPlaneWrapper(env, channel_first=False)
print(env.observation_space("player_0"))
env = ChannelLastToFirstWrapper(env)
print(env.observation_space("player_0"))

env = TicTacToeConfig().make_env()
env.reset()
env.step(0)
env.step(6)
env.step(2)

state, reward, terminated, truncated, info = env.last()
prediction = agent.predict(state, info, env)
print("MCTS Prediction", prediction)
initial_inference = agent.predict_single_initial_inference(state, info)
print("Initial Value", initial_inference[0])
print("Initial Policy", initial_inference[1])
for move in info["legal_moves"]:
    reccurent_inference = agent.predict_single_recurrent_inference(
        initial_inference[2], move
    )
    print("Move", move)
    print("Reccurent Value", reccurent_inference[2])
    print("Reccurent Reward", reccurent_inference[0])
    print("Reccurent Policy", reccurent_inference[3])


action = agent.select_actions(prediction).item()

selected_actions = {i: 0 for i in range(agent.num_actions)}
for i in range(100):
    selected_actions[agent.select_actions(prediction).item()] += 1
print(selected_actions)
print("Action", action)
env.step(action)
state, reward, terminated, truncated, info = env.last()
prediction = agent.predict(state, info, env)
print("MCTS Prediction", prediction)
initial_inference = agent.predict_single_initial_inference(state, info)
print("Initial Value", initial_inference[0])
print("Initial Policy", initial_inference[1])
for move in info["legal_moves"]:
    reccurent_inference = agent.predict_single_recurrent_inference(
        initial_inference[2], move
    )
    print("Move", move)
    print("Reccurent Value", reccurent_inference[2])
    print("Reccurent Reward", reccurent_inference[0])
    print("Reccurent Policy", reccurent_inference[3])


action = agent.select_actions(prediction).item()
print("Action", action)
env.step(action)
state, reward, terminated, truncated, info = env.last()
prediction = agent.predict(state, info, env)
print("MCTS Prediction", prediction)
initial_inference = agent.predict_single_initial_inference(state, info)
print("Initial Value", initial_inference[0])
print("Initial Policy", initial_inference[1])

for move in info["legal_moves"]:
    reccurent_inference = agent.predict_single_recurrent_inference(
        initial_inference[2], move
    )
    print("Move", move)
    print("Reccurent Value", reccurent_inference[2])
    print("Reccurent Reward", reccurent_inference[0])
    print("Reccurent Policy", reccurent_inference[3])

action = agent.select_actions(prediction).item()
print("Action", action)
# env.step(action)
# state, reward, terminated, truncated, info = env.last()
# prediction = agent.predict(state, info, env)
# print("MCTS Prediction", prediction)
# initial_inference = agent.predict_single_initial_inference(state, info)
# print("Initial Value", initial_inference[0])
# print("Initial Policy", initial_inference[1])
# action = agent.select_actions(prediction).item()
# print("Action", action)
# env.step(action)

In [ ]:
from copy import deepcopy
from math import log, sqrt, inf
import copy
import math
import numpy as np


class Node:
    def __init__(self, prior_policy):
        self.visits = 0
        self.to_play = -1
        self.prior_policy = prior_policy
        self.value_sum = 0
        self.children = {}
        self.hidden_state = None
        self.reward = 0

    def expand(self, legal_moves, to_play, policy, hidden_state, reward):
        self.to_play = to_play
        self.reward = reward
        self.hidden_state = hidden_state
        # print(legal_moves)
        policy = {a: policy[a] for a in legal_moves}
        policy_sum = sum(policy.values())

        for action, p in policy.items():
            self.children[action] = Node((p / (policy_sum + 1e-10)).item())

    def expanded(self):
        return len(self.children) > 0

    def value(self):
        if self.visits == 0:
            return 0
        return self.value_sum / self.visits

    def add_noise(self, dirichlet_alpha, exploration_fraction):
        actions = list(self.children.keys())
        noise = np.random.dirichlet([dirichlet_alpha] * len(actions))
        frac = exploration_fraction
        for a, n in zip(actions, noise):
            self.children[a].prior_policy = (1 - frac) * self.children[
                a
            ].prior_policy + frac * n

    def select_child(self, min_max_stats, pb_c_base, pb_c_init, discount, num_players):
        # Select the child with the highest UCB
        child_ucbs = [
            self.child_ucb_score(
                child, min_max_stats, pb_c_base, pb_c_init, discount, num_players
            )
            for action, child in self.children.items()
        ]
        print("Child UCBs", child_ucbs)
        action_index = np.random.choice(
            np.where(np.isclose(child_ucbs, max(child_ucbs)))[0]
        )
        action = list(self.children.keys())[action_index]
        return action, self.children[action]

    def child_ucb_score(
        self, child, min_max_stats, pb_c_base, pb_c_init, discount, num_players
    ):
        pb_c = log((self.visits + pb_c_base + 1) / pb_c_base) + pb_c_init
        pb_c *= sqrt(self.visits) / (child.visits + 1)

        prior_score = pb_c * child.prior_policy
        if child.visits > 0:
            value_score = min_max_stats.normalize(
                child.reward
                + discount
                * (
                    child.value() if num_players == 1 else -child.value()
                )  # (or if on the same team)
            )
        else:
            value_score = 0.0

        # check if value_score is nan
        assert (
            value_score == value_score
        ), "value_score is nan, child value is {}, and reward is {},".format(
            child.value(),
            child.reward,
        )
        assert prior_score == prior_score, "prior_score is nan"
        print("Prior Score", prior_score)
        print("Value Score", value_score)
        return prior_score + value_score
        # return value_score

In [ ]:
agent.config.num_simulations = 10

In [ ]:
import collections
from typing import Optional

from pyparsing import List

MAXIMUM_FLOAT_VALUE = float("inf")


class MinMaxStats(object):
    def __init__(
        self, known_bounds: Optional[List[float]]
    ):  # might need to say known_bounds=None
        self.max = known_bounds[1] if known_bounds else MAXIMUM_FLOAT_VALUE
        self.min = known_bounds[0] if known_bounds else -MAXIMUM_FLOAT_VALUE

    def update(self, value: float):
        self.max = max(self.max, value)
        self.min = min(self.min, value)

    def normalize(self, value: float) -> float:
        print("Initial value", value)
        if self.max > self.min:
            # We normalize only when we have at a max and min value
            print("normalized value", (value - self.min) / (self.max - self.min))
            return (value - self.min) / (self.max - self.min)
        return value

    def __repr__(self):
        return f"min: {self.min}, max: {self.max}"

In [ ]:
from utils.utils import get_legal_moves

# from muzero.muzero_mcts import Node
# from muzero.muzero_minmax_stats import MinMaxStats


root = Node(0.0)
_, policy, hidden_state = agent.predict_single_initial_inference(
    state,
    info,
)
print("root policy", policy)
legal_moves = get_legal_moves(info)[0]
to_play = env.agents.index(env.agent_selection)
root.expand(legal_moves, to_play, policy, hidden_state, 0.0)
print("expanded root")
min_max_stats = MinMaxStats(agent.config.known_bounds)

for _ in range(agent.config.num_simulations):
    print("at root")
    node = root
    search_path = [node]
    to_play = env.agents.index(env.agent_selection)

    # GO UNTIL A LEAF NODE IS REACHED
    while node.expanded():
        print("selecting child")
        action, node = node.select_child(
            min_max_stats,
            agent.config.pb_c_base,
            agent.config.pb_c_init,
            agent.config.discount_factor,
            agent.config.game.num_players,
        )
        print("Selected action", action)
        # THIS NEEDS TO BE CHANGED FOR GAMES WHERE PLAYER COUNT DECREASES AS PLAYERS GET ELIMINATED, USE agent_selector.next() (clone of the current one)
        to_play = (to_play + 1) % agent.config.game.num_players
        search_path.append(node)
    parent = search_path[-2]
    reward, hidden_state, value, policy = agent.predict_single_recurrent_inference(
        parent.hidden_state,
        action,  # model=model
    )
    reward = reward.item()
    value = value.item()
    print("leaf value", value)
    print("leaf reward", reward)

    node.expand(
        list(range(agent.num_actions)),
        to_play,
        policy,
        hidden_state,
        (
            reward  # if self.config.game.has_intermediate_rewards else 0.0
        ),  # for board games and games with no intermediate rewards
    )

    for node in reversed(search_path):
        node.value_sum += value if node.to_play == to_play else -value
        node.visits += 1
        min_max_stats.update(
            node.reward
            + agent.config.discount_factor
            * (node.value() if agent.config.game.num_players == 1 else -node.value())
        )
        value = (
            -node.reward
            if node.to_play == to_play and agent.config.game.num_players > 1
            else node.reward
        ) + agent.config.discount_factor * value

    visit_counts = [(child.visits, action) for action, child in root.children.items()]

print(visit_counts)